## Libraries

In [1]:
import pandas as pd
import re

## Open csv

In [2]:
df = pd.read_csv("../output/frenchCropUsage_20210525/d2kab/conjunctions.csv", delimiter="\t", encoding='utf-8')
df[:3]

,bsv,context,full_expression,match_with_fcu,context.1,location,features
0,20180911_bsv_grandes_cultures-26_cle0f15a8,1 Pluri-annuelles (,"prairies, jachères","prairies, jachères",) 5 Colza/Blé Maïs/,4028-4046,"type=N_PUN_N, form=prairies, jachères"
1,20181106_bsv_grandes_cultures_34_cle08159b,7 feuilles Enquête,Tournesol et oiseaux,Tournesol,Céréales à paille L,675-695,"type=N_KON_N, form=Tournesol et oiseaux"
2,20181106_bsv_grandes_cultures_34_cle08159b,L Baris Du point de,"vue des maladies, les colzas",colzas,sont très sains. Qu,9623-9651,"type=N_PUN_N, form=vue des maladies, les colzas"


## Extracted conjunctions 

In [3]:
articles = ["le", "la", "les"]
values = df["full_expression"].values.tolist()


conjunctions_list = []
for line in values:
    line = line.lower()
    line = line.replace(' et ', ',')
    line = line.replace('(', ',')
    line = line.replace(')', ',')
    line = line.replace("l'", '')
    lines = line.split(',')
    lines_clean = []
    for l in lines:
        le = l.split()
        le = [word for word in le if word not in articles]
        lines_clean.append(" ".join(le))
    lines_clean = [word.strip() for word in lines_clean]
    conjunctions_list.append(lines_clean)
conjunctions_list

[['prairies', 'jachères'],
 ['tournesol', 'oiseaux'],
 ['vue des maladies', 'colzas'],
 ['blés', 'orges'],
 ['compte', 'stade', 'vigueur du colza'],
 ['colza', 'tournesol', 'soja', 'pois'],
 ['blés', 'stade'],
 ['colza', 'tournesol', 'pois'],
 ['contamination du colza', 'application'],
 ['pailles', 'chaumes'],
 ['pois', 'féverole'],
 ['blés', 'apparition des barbes'],
 ['effet', 'colzas'],
 ['effet', 'colza'],
 ['colza', 'tournesol', 'soja', 'pois'],
 ['sol', 'maïs'],
 ['semaine', 'stades des blés'],
 ['colza', 'tournesol', 'soja', 'pois'],
 ['semaine', 'maïs'],
 ['mildiou du tournesol', 'maladie'],
 ['colza', 'altises'],
 ['développement des colzas', 'seuil de nuisibilité'],
 ['développement des colzas', 'seuil de nuisibilité'],
 ['couleur du colza', 'qualité de enracinement', 'disponibilité'],
 ['colzas', 'pucerons'],
 ['colza', 'larve de teigne'],
 ['colza', 'mouche du chou'],
 ['parcelles de blé', "d'orge"],
 ['blé', 'escourgeon'],
 ['blé', 'orge'],
 ['colza', 'stade'],
 ['colza', 

## FCU terms

In [4]:
matches = df["match_with_fcu"].values.tolist()

fcu_list = []
for line in matches:
    line = line.lower()
    lines = line.split(', ')
    lines = [word.strip() for word in lines]
    fcu_list.append(lines)
fcu_list

[['prairies', 'jachères'],
 ['tournesol'],
 ['colzas'],
 ['blés'],
 ['colza'],
 ['colza', 'tournesol', 'soja', 'pois'],
 ['blés'],
 ['colza', 'tournesol', 'pois'],
 ['colza'],
 ['pailles'],
 ['pois', 'féverole'],
 ['blés'],
 ['colzas'],
 ['colza'],
 ['colza', 'tournesol', 'soja', 'pois'],
 ['maïs'],
 ['blés'],
 ['colza', 'tournesol', 'soja', 'pois'],
 ['maïs'],
 ['tournesol'],
 ['colza'],
 ['colzas'],
 ['colzas'],
 ['colza'],
 ['colzas'],
 ['colza'],
 ['colza', 'chou'],
 ['blé'],
 ['blé', 'escourgeon'],
 ['blé', 'orge'],
 ['colza'],
 ['colza'],
 ['colza'],
 ['colza'],
 ['blé'],
 ['céréale'],
 ['orge'],
 ['maïs'],
 ['champignons'],
 ['colza'],
 ['colza'],
 ['colza'],
 ['blé', 'triticale'],
 ['blé', 'triticale'],
 ['blés', 'triticale'],
 ['triticale'],
 ['colza'],
 ['colza'],
 ['colza', 'céréales', 'fourrages'],
 ['tournesol', 'colza'],
 ['colza'],
 ['escourgeons'],
 ['gousses'],
 ['colza'],
 ['sorgho'],
 ['escourgeons'],
 ['pois de printemps'],
 ['tournesol'],
 ['tournesol', 'soja'],
 [

## Propositions

In [5]:
def Diff(li1, li2):
    return list(set(li1) - set(li2)) 

diffs = []
for i, line in enumerate(conjunctions_list):
    diff = Diff(line, fcu_list[i])
    diff = ",\n".join(str(x) for x in diff)
    diffs.append(diff)

## Add propositions 

In [6]:
df.insert(4,'proposition',diffs)
df[:3]

,bsv,context,full_expression,match_with_fcu,proposition,context.1,location,features
0,20180911_bsv_grandes_cultures-26_cle0f15a8,1 Pluri-annuelles (,"prairies, jachères","prairies, jachères",,) 5 Colza/Blé Maïs/,4028-4046,"type=N_PUN_N, form=prairies, jachères"
1,20181106_bsv_grandes_cultures_34_cle08159b,7 feuilles Enquête,Tournesol et oiseaux,Tournesol,oiseaux,Céréales à paille L,675-695,"type=N_KON_N, form=Tournesol et oiseaux"
2,20181106_bsv_grandes_cultures_34_cle08159b,L Baris Du point de,"vue des maladies, les colzas",colzas,vue des maladies,sont très sains. Qu,9623-9651,"type=N_PUN_N, form=vue des maladies, les colzas"


In [7]:
df.to_csv("../output/frenchCropUsage_20210525/d2kab/conjunctions_final.csv", sep = '\t', encoding = 'utf-8', index=True)